In [1]:
import pandas as pd

In [121]:
df = pd.read_csv("data/stream_e.csv", sep = "ef5r64i", engine = "python")
print(df.columns, df.shape)


Index(['Texto', 'IdUsuario', 'FechaCreacion', 'IdTweet', 'NombreUsuario',
       'FechaCreacionTweet', 'Hashtags', 'Url'],
      dtype='object') (1268, 8)


In [122]:
#Creating the dict with unique users and a list for the tweets they have written
corpus = {}
for i in range(df.shape[0]):
    if df.iloc[i]["NombreUsuario"] in corpus:
        corpus[df.iloc[i]["NombreUsuario"]].append(df.iloc[i]["Texto"])
    else:
        corpus[df.iloc[i]["NombreUsuario"]] = [df.iloc[i]["Texto"]]
len(corpus)

1189